In [1]:
"""
A script to annotate the dialog data with sentiment 
"""
#This is how to install packages
import sys
#!{sys.executable} -m pip install vaderSentiment
#!{sys.executable} -m pip install pymongo

In [2]:
"""
Some examples of the code
"""
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
#from vaderSentiment import SentimentIntensityAnalyzer

# --- examples -------
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all",  # Capitalized negation
             "I hate this shit",
             "Not nice"
             ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.------------------------ {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.----------------------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'co

In [3]:
# here is connection to the db
# set up connection to the MongoDB: sudo service mongod start (27017 is the default port)
from pymongo import MongoClient
import json

class Mongo_Connector():
    '''
    Wrapper class for some of the pymongo functions: http://api.mongodb.com/python/current/tutorial.html
    '''

    def __init__(self, db_name):
        # spin up database
        self.mongo_client = MongoClient()
        self.db = self.mongo_client[db_name]
        print("Connection success.")
    
    def count_all_docs(self, collection_name):
        count = self.db[collection_name].count_documents({})
        print ("%d dialogues in %s" % (count, collection_name))
    
    def bulk_load(self, collection_name, json_file_path, chunk_size=1000):
        '''
        Imports a big dataset from a single JSON file
        '''
        # load JSON
        with open(json_file_path) as f:
            all_docs = json.loads(f.read())
        
        n_docs = len(all_docs)
        print("Loading %d dialogues"%n_docs)
                
        # iterate over the docs
        chunk = []
        for _id, dialogue in all_docs.items():
            dialogue['_id'] = _id
            chunk.append(dialogue)
            if len(chunk) == chunk_size:
                # insert chunk   
                self.db[collection_name].insert_many(chunk)
                chunk = []
        # insert the last chunk
        if chunk:
            self.db[collection_name].insert_many(chunk)

        # show the doc counter
        self.count_all_docs(collection_name)


db_name = 'cm'
mongo = Mongo_Connector(db_name)

Connection success.


In [105]:
# here to annotate the dstc6 messages

collection = mongo.db['dstc6']
cursor = collection.find({})
#varI = cursor[0]
#print(varI)

{'_id': ObjectId('5c19081fc948caf3a61c96fe'), 'group-id': '', 'speaker-id': '336347236475', 'turns': [{'speaker': 'U', 'annotations': [], 'utterance': 'Hi Alex', 'time': '', 'turn-index': 0, 'sentiment': 'Ne'}, {'speaker': 'S', 'annotations': [{'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'X', 'annotator-id': '43374108'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '34557302'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '37118552'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '43226197'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '1954947'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '4338784'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '33305451'}, {'comment': '', 'ungrammatical-sentence': '', 'breakdown': 'O', 'annotator-id': '18482369'}, {'comment': ''

In [104]:
count = 0
for document in cursor:
    doc = document
    id = doc['_id']
    for i in range(len(doc['turns'])):
        ut = doc['turns'][i]
        vs = analyzer.polarity_scores(doc['turns'][i]['utterance'])
        #print ("polarity:")
        if vs['neg'] > vs['pos'] and vs['neg'] > vs['neu']: 
            doc['turns'][i]['sentiment'] = 'N'
            print ("neg")
            print (vs)
        elif vs['pos'] > vs['neg'] and vs['pos'] > vs['neu']:
            doc['turns'][i]['sentiment'] = 'P'
            #document.update_one(ut, {"$set" : {"Sentiment":"P"}})
            print ("pos")
        else:
            doc['turns'][i]['sentiment'] = 'Ne'
    collection.update_one({'_id': id}, {"$set": doc}, upsert=True)

pos
pos
pos
neg
{'neg': 0.557, 'neu': 0.443, 'pos': 0.0, 'compound': -0.8418}
neg
{'neg': 0.523, 'neu': 0.477, 'pos': 0.0, 'compound': -0.897}
pos
neg
{'neg': 0.535, 'neu': 0.465, 'pos': 0.0, 'compound': -0.6808}
pos
pos
pos
pos
pos
neg
{'neg': 0.567, 'neu': 0.433, 'pos': 0.0, 'compound': -0.3865}
neg
{'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}
neg
{'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}
pos
neg
{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.7269}
neg
{'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}
pos
pos
pos
pos
pos
pos
neg
{'neg': 0.508, 'neu': 0.492, 'pos': 0.0, 'compound': -0.4767}
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
neg
{'neg': 0.688, 'neu': 0.312, 'pos': 0.0, 'compound': -0.296}
pos
pos
pos
neg
{'neg': 0.756, 'neu': 0.244, 'pos': 0.0, 'compound': -0.4767}
pos
pos
pos
pos
pos
pos
neg
{'neg': 0.592, 'neu': 0.408, 'pos': 0.0, 'compound': -0.7003}
neg
{'neg': 0.697, 'neu': 0.303, 'pos': 0.0, 'compound': -0.6808}
pos

In [6]:
# the msdialog sentiment annotation

collection = mongo.db['msdialog']
#for turn in sample_doc['utterances']:
#    print ("%s: %s"% (turn['actor_type'], turn['utterance']))
cursor = collection.find({})

In [7]:
count = 0
for document in cursor:
    doc = document
    id = doc['_id']
    for i in range(len(doc['utterances'])):
        ut = doc['utterances'][i]
        vs = analyzer.polarity_scores(doc['utterances'][i]['utterance'])
        #print ("polarity:")
        if vs['neg'] > vs['pos'] and vs['neg'] > vs['neu']: 
            doc['utterances'][i]['sentiment'] = 'N'
            #print ("neg")
        elif vs['pos'] > vs['neg'] and vs['pos'] > vs['neu']:
            doc['utterances'][i]['sentiment'] = 'P'
            #document.update_one(ut, {"$set" : {"Sentiment":"P"}})
            #print ("pos")
        else:
            doc['utterances'][i]['sentiment'] = 'Ne'
    collection.update_one({'_id': id}, {"$set": doc}, upsert=True)
    print("one doc done")

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

...

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done

one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done
one doc done